In [1]:
import pandas as pd

In [2]:
ref = pd.read_excel(r'C:\Users\sgasmi\Desktop\Extractions\Query_20210112093258.xlsx')

In [25]:
ref["Level fille"] = ref["Code type d'unité de tr."].apply(lambda x: x[1:])
ref["Level mère"] = ref.iloc[:, 8]
ref["Level mère"] = ref["Level mère"].fillna("00")
ref["Level mère"] = ref["Level mère"].apply(lambda x: x[1:])
ref["Level mère"] = ref["Level mère"].astype("int64")
ref["Level fille"] = ref["Level fille"].astype("int64")

def check_level(level_fille, level_mere):
    if level_fille>0:
        if (level_fille-1) != level_mere:
            return 1
        else:
            return 0
    else:
        return 0

ref["Anomalie niveau"] = ref.apply(lambda x: check_level(x["Level fille"],x["Level mère"]), axis=1)
ref.loc[ref["Matricule RH"].isna()==True, "RH Manquant"] = 1
ref.loc[ref["Matricule Manager"].isna()==True, "Manager Manquant"] = 1
anomalie_niveau = ref.loc[(ref["Anomalie niveau"]==1) & (ref["Nom de l'unité de travail"] != "Racine")]
anomalie_RH = ref.loc[ref["RH Manquant"]==1]
anomalie_Manager = ref.loc[ref["Manager Manquant"]==1]

In [28]:
print(anomalie_niveau)

     Code de l'unité de travail Nom de l'unité de travail  \
0                         10199                  IBNDTP-F   
1253                       7561                  IBNDIP-F   
1254                       7562                 IBNDTSD-F   
1255                       7563                 IBNDTPR-F   
1256                       7564                  IBNDTV-F   
1257                       7565                  IBNDTS-F   
1258                       7566                 IBNDTIS-F   
1259                       7567                 IBNDTNM-F   
1260                       7568                 IBNDTTD-F   
1261                       7569                 IBNDTNS-F   
1262                       7570                IBNNETEC-F   

     Code type d'unité de tr. Objectifs Date de début  Date de fin  \
0                          L5        IB    1951-01-01          NaN   
1253                       L6        IB    1951-01-01          NaN   
1254                       L6        IB    1951-01-01    

In [30]:
if len(anomalie_niveau)>0 or len(anomalie_RH)>0 or len(anomalie_Manager)>0 :
    anomalie_niveau.to_excel(r'C:\Users\sgasmi\Desktop\Export Python\Anomalie UO.xlsx', index=False)
    with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\Anomalie UO.xlsx') as writer:  
        anomalie_niveau.to_excel(writer, sheet_name='Anomalies_niveau', index=False)
        anomalie_RH.to_excel(writer, sheet_name='Anomalies_RH', index=False)
        anomalie_Manager.to_excel(writer, sheet_name='Anomalie_Manager', index=False)
assert len(anomalie_niveau)==0 and len(anomalie_RH)==0 and len(anomalie_Manager)==0  , print("Incohérence dans le référentiel")

Incohérence dans le référentiel


AssertionError: None

In [5]:
ref = ref.loc[ref["Code type d'unité de tr."]!="00"] # Suppression du niveau 0
ref.reset_index(drop=True, inplace=True)

In [6]:
col = ["1","2","3","4","5","6","7","8"]
result = pd.DataFrame(columns=col)

In [7]:
print(ref.loc[ref["Code de l'unité de travail"]=="ROOT"])

Empty DataFrame
Columns: [Code de l'unité de travail, Nom de l'unité de travail, Code type d'unité de tr., Objectifs, Date de début, Date de fin, Description, Code unité de tr. mère, Code type d'unité de tr..1, Date de début rattachement, Date de fin rattachement, Code externe UT, Code de l'unité de travail.1, Matricule Manager, Nom Manager, Date de début Manager, Prénom Manager, Matricule RH, Nom RH, Date de début RH, Prénom RH, Level fille, Level mère, Anomalie niveau, RH Manquant, Manager Manquant]
Index: []

[0 rows x 26 columns]


In [8]:
for i, val in enumerate(ref.iterrows()):
    col_cible = ref.at[i, "Code type d'unité de tr."][1]
    # Alimentation de la première colonne
    result.at[i, col_cible] = ref.at[i,"Nom de l'unité de travail"]
    #Alimentation des autres colonnes
    cmere = ""

    while int(col_cible)>1: 
        if cmere== "":
            cmere=ref.at[i,"Code unité de tr. mère"]
        try:
            nuo = ref.loc[ref["Code de l'unité de travail"]==cmere,"Nom de l'unité de travail"].iloc[0]
        except:
            print(ref.at[i,"Nom de l'unité de travail"])
        col_cible= str(int(col_cible)-1)
        result.at[i, col_cible] = nuo
        cmere = ref.loc[ref["Code de l'unité de travail"]==cmere,"Code unité de tr. mère"].iloc[0]
        
     
        #result.at[i, col_cible-1] = ref.loc[ref["Nom de l'unité de travail"]==ref.at[i,"Code unité de tr. mère"]]
result = result.merge(ref[["Code de l'unité de travail","Nom de l'unité de travail","Code type d'unité de tr.", "Description"]], left_index=True, right_index=True)

IBNDIP-F


IndexError: single positional indexer is out-of-bounds

In [ ]:
# Réorganisation des colonnes

col = result.columns
l1 = list(col[-3:])
l2 = list(list(col[:-3]))
col = l1 + l2

result = result[col]
print(result.columns)
result.columns=["Nom de l'unité de travail","Code type d'unité de tr.", "Description",
       "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8", "Code de l'unité de travail", ]

result = result[["Code de l'unité de travail","Nom de l'unité de travail", "Code type d'unité de tr.", "Description",
       "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8"]]
#print(list(col[-2:] + list(col[:-2]))

### Ajout de l'UO mère

In [ ]:
mere = ref[["Code de l'unité de travail", "Code unité de tr. mère"]]
result = result.merge(mere, on="Code de l'unité de travail", how="left")
mere = ref[["Code de l'unité de travail", "Nom de l'unité de travail"]]
result = result.merge(mere, left_on="Code unité de tr. mère", right_on="Code de l'unité de travail", how="left")
result = result.drop(columns=["Code unité de tr. mère", "Code de l'unité de travail_y"], axis=1)
result = result.rename(columns={"Nom de l'unité de travail_y": "UO mère", "Code de l'unité de travail_x":"Code de l'unité de travail", "Nom de l'unité de travail_x": "Nom de l'unité de travail"})
result.loc[result["UO mère"].isna()==True, "UO mère"] = "ROOT"

In [ ]:
result.to_excel(r'C:\Users\sgasmi\Desktop\Export Python\UO_Org.xlsx', index=False)